In [ ]:
from keras.models import load_model

model=load_model('final_90.h5')


In [ ]:
from pickle import dump
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model, load_model
from keras.applications.vgg16 import preprocess_input
import os
import tokenize
import string
from numpy import array
from pickle import load
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu
import urllib.request

In [ ]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=1)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [ ]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 40

In [ ]:
#load dataset and use features 
def extract(path):
    model = VGG19()  #FROM https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
    model.layers.pop()#removes last added layer
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    features = dict()
    files=os.listdir(path)
    for i in range(0,len(files)):#0 to len(files)-7 in case of data as last 7 files are .txt files
        filename = path + '/' + files[i]
        print(filename)
        image = load_img(filename, target_size=(224, 224,3))#Alex net Size
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = files[i].split('.')[0]
        features[image_id] = feature
    return features

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
i=0
while(True):
    photo=extract('test')['test']
    description = generate_desc(model, tokenizer, photo, max_length)
    print(description)
    break
    # Display the resulting frame
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
count=0
import cv2
while(1):
    try:
        imgResp=urllib.request.urlopen('http://192.168.43.80/snapshot.jpg')
        imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
        img=cv2.imdecode(imgNp,-1)
           #image=detect_image(img,yolo,all_classes)
        path='test/'
        cv2.imwrite(os.path.join(path,'test.jpg'),img)
        photo=extract('test')['test']
        description = generate_desc(model, tokenizer, photo, max_length)
        print(description)
    except:
        print('error')


In [ ]:
description1=description[1:len(description)-1]
print(description1)
text=" "
for i in description1:
    text=text+i
print(text)

In [ ]:
import serial
import time
import requests
import json
firebase_url = 'https://iot-v5-0.firebaseio.com/'
#Connect to Serial Port for communication
ser = "hi"
#Setup a loop to send Temperature values at fixed intervals
#in seconds
fixed_interval = 1
i=0
while 1:
  try:
    #temperature value obtained from Arduino + LM35 Temp Sensor          
    data_c = description
    print ((data_c))
    
    #insert record
    data = {'value':(data_c)}
    result = requests.post(firebase_url + '/' +'/data/1.json' ,data=json.dumps((data)))
    print('Record inserted. Result Code = ' + str(result.status_code) + ',' + result.text)
    time.sleep(fixed_interval)
    i=i+1
  except IOError:
    print('Error! Something went wrong.')
  time.sleep(fixed_interval)

